# Introduksjon

## Forfatter(e)
1. Magne Furuholmen Myhren . Basert på en rekke sas-filer, tidligere drop-lister og spesifikk kompetanse fra kollegaer (30.09.22).

## Forklaring

Denne notebooken bruker data fra SFU, KLASS og DYNAREV til å sette sammen _droplister_ til hvert skjema spesialisthelsetjenesten sender ut. Droplister er .csv-filer som inneholder en oversikt over alle foretak som har rapporteringsansvar, enten bare for seg selv eller for undervirksomheter. 

For at filene skal passe med KOSTRA-IT sine programmer, må de ha en bestemt utforming hvor kolonnenes rekkefølge har betydning og noen kolonner har særskilte krav når det kommer linjeskift og tegnsetting.

Notebooken er en fullverdig erstatning for et helt løp tidligere gjort i SAS.

## Følgende datakilder må være oppdatert

- [x] Standardtabell 603, 604 og 605 i KLASS
- [x] SFU (DYNAREV delregister 24xx og 19377xx)

## Oversikt over skjema
```Variabel``` er variabelnavnet som peker til droplisten.

  Variabel      |  Skjemanavn      |  Forklaring                                       
----------------|------------------|-------------------------------------------------------
  skj0X         |  0X              |  Resultatregnskap for RFH og HF                       
  skj0Y         |  0Y              |  Balanseregnskap for RFH og HF                        
  skj38O        |  HELSE38O        |  TSB for offentlige helseforetak                      
  skj38P        |  HELSE38P        |  TSB for private foretak                              
  skj39         |  HELSE39         |  Resultatregnskap for private                         
  skj40         |  HELSE40         |  Kontantstrømoppstilling                              
  skj41         |  HELSE41         |  Private spesialister med driftsavtale                
  skj44O        |  HELSE44O        |  Psykisk helsevern for voksne, offentlige             
  skj44P        |  HELSE44P        |  Psykisk helsevern for voksne, private                
  skj45O        |  HELSE45O        |  Psykisk helsevern for barn og unge, offentlige       
  skj45P        |  HELSE45P        |  Psykisk helsevern for barn og unge, private          
  skj46O        |  46O             |  Somatiske sykehus, offentlige                        
  skj46P        |  HELSE46P        |  Somatiske sykehus, private                           
  skj47         |  HELSE47         |  Somatiske institusjoner                              
  skj48         |  HELSE48         |  Praksiskonsulentordningen                            

# Setup

## Importerer pakker og setter innstillinger

In [19]:
import numpy as np
import pandas as pd
import cx_Oracle
from db1p import query_db1p
import getpass
import os
import requests
from pandas.io.json import json_normalize #package for flattening json in pandas df

# Fjerner begrensning på antall rader og kolonner som vises av gangen
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

# Unngå standardform i output
pd.set_option('display.float_format', lambda x: '%.0f' % x)

## Velger årgang og delregister

NB! For å lagre filene til angitt sti må `lagre_filer` settes til `True`.

In [20]:
aargang = 2022
siste_to_siffer_aargang = str(aargang)[-2:]

aar_for = aargang - 1            # året før
siste_to_siffer_aar_for = str(aar_for)[-2:]

dato_idag = pd.Timestamp("today").strftime("%d%m%y")

###############################################################################
lagre_filer = False              # sett til True, hvis du vil lagre droplistene
###############################################################################
stamme_sti = "/ssb/stamme01/fylkhels/speshelse/felles/droplister/"

In [21]:
# lagrer i mappe sortert på aargang og dagens dato. Lager ny mappe hvis
# den allerede ikke eksisterer
if lagre_filer:
    if not os.path.exists(stamme_sti + str(aargang)):
        os.makedirs(stamme_sti + str(aargang))

    if not os.path.exists(stamme_sti + str(aargang) + "/" + dato_idag):
        os.makedirs(stamme_sti + str(aargang) + "/" + dato_idag)

    sti_til_lagring = stamme_sti + str(aargang) + "/" + dato_idag + "/"

## Tilgang oracle

In [22]:
conn = cx_Oracle.connect(getpass.getuser()+"/"+getpass.getpass(prompt='Oracle-passord: ')+"@DB1P")

Oracle-passord:  ········


# Henter data

##  fra KLASS

### Offentlige helseforetak

In [24]:
URL = f'http://data.ssb.no/api/klass/v1/classifications/603/codes.json?from={aargang}-01-01&includeFuture=True'
r = requests.get(url = URL)
offhelse_df = pd.read_json(r.text)
offhelse_df = pd.json_normalize(offhelse_df['codes'])

RHF_kode_klass = offhelse_df.query('level=="1"')[['code','name']]
RHF_kode_klass = RHF_kode_klass.rename(columns = {'code':'HELSEREGION', 'name': 'RHF'})

lvl2 = offhelse_df.query("level == '2'")
RHF_kode_klass = pd.merge(RHF_kode_klass, lvl2, how="left", left_on="HELSEREGION", right_on="parentCode")
RHF_kode_klass = RHF_kode_klass.rename(columns = {'code': 'ORGNR_FORETAK'})
RHF_kode_klass = RHF_kode_klass[['HELSEREGION', 'RHF', 'ORGNR_FORETAK']]

lvl2 = offhelse_df.query("level == '2'")
lvl3 = offhelse_df.query("level == '3'")
lvl2 = lvl2[['code', 'parentCode', 'name']]
lvl3 = lvl3[['code', 'parentCode', 'name']]

lvl2 = lvl2.rename(columns = {'parentCode':'HELSEREGION', 'name': 'RHF'})
lvl3 = lvl3.rename(columns = {'code':'ORGNR_FORETAK', 'name': 'NAVN_KLASS'})

In [25]:
HF = pd.merge(lvl3, lvl2, how="left", left_on="parentCode", right_on="code")
HF = HF[['ORGNR_FORETAK', 'NAVN_KLASS', 'HELSEREGION', 'RHF']]

RHF_kode_klass['RHF'] = RHF_kode_klass['RHF'] + " RHF"
RHF_kode_klass['NAVN_KLASS'] = RHF_kode_klass['RHF']

### Private helseinstitusjoner med oppdrags- og bestillerdokument

In [26]:
URL = f'http://data.ssb.no/api/klass/v1/classifications/604/codes.json?from={aargang}-01-01&includeFuture=True'
r = requests.get(url = URL)
privhelse_df = pd.read_json(r.text)
privhelse_df = pd.json_normalize(privhelse_df['codes'])

temp = privhelse_df.query("level == '2'")
temp = temp[['code', 'parentCode', 'name']]
temp = temp.rename(columns = {'code':'ORGNR_FORETAK','parentCode':'HELSEREGION', 'name': 'NAVN_KLASS'})
temp = pd.merge(temp, RHF_kode_klass, how="left")

rapporteringsenheter = pd.concat([HF,temp])
rapporteringsenheter = pd.concat([rapporteringsenheter,RHF_kode_klass])

### Regionale og felleseide støtteforetak i spesialisthelsetjenesten

In [27]:
URL = f'http://data.ssb.no/api/klass/v1/classifications/605/codes.json?from={aargang}-01-01&includeFuture=True'
r = requests.get(url = URL)
regfel_df = pd.read_json(r.text)
regfel_df = pd.json_normalize(regfel_df['codes'])

lvl2o = regfel_df.query("level == '2' & parentCode != '99'")
lvl3o = regfel_df.query("level == '3'")
lvl2o = lvl2o[['code', 'parentCode', 'name']]
lvl3o = lvl3o[['code', 'parentCode', 'name']]
lvl2o = lvl2o.rename(columns = {'parentCode':'HELSEREGION',
                                'name': 'RHF'})
lvl3o = lvl3o.rename(columns = {'code': 'ORGNR_FORETAK',
                                'name': 'NAVN_KLASS'})

temp = pd.merge(lvl3o, lvl2o, how="left", left_on="parentCode", right_on="code")
temp = temp[['ORGNR_FORETAK', 'NAVN_KLASS', 'HELSEREGION', 'RHF']]

In [28]:
rapporteringsenheter = pd.concat([rapporteringsenheter,temp])

temp = regfel_df.query("level == '2' & parentCode == '99'")
temp = temp.rename(columns = {'code':'ORGNR_FORETAK', 'name': 'NAVN_KLASS', 'parentCode': 'HELSEREGION'})
temp['RHF'] = "FELLESEIDE STØTTEFORETAK"
temp = temp[['ORGNR_FORETAK', 'NAVN_KLASS', 'HELSEREGION', 'RHF']]

In [29]:
rapporteringsenheter = pd.concat([rapporteringsenheter,temp])

### Limer sammen all data fra KLASS

In [30]:
alle_foretak_virk = pd.concat([offhelse_df, privhelse_df, regfel_df])
alle_foretak_virk = alle_foretak_virk[["code", "parentCode", "level", "name"]]

## VOF: liste over alle helseforetak

In [31]:
rapporteringsenheter_uten_RHF = rapporteringsenheter.query('~NAVN_KLASS.str.endswith("RHF")',engine="python")

In [32]:
r_orgnr = rapporteringsenheter_uten_RHF.ORGNR_FORETAK.to_numpy()

In [33]:
def lag_sql_str(arr):
    s = "("
    for nr in arr:
        s += "'" + str(nr) + "',"
    s = s[:-1] + ")"
    return s

sql_str = lag_sql_str(r_orgnr)

sporring_for = f"""
    SELECT FORETAKS_NR, ORGNR, NAVN
    FROM DSBBASE.SSB_FORETAK
    WHERE STATUSKODE = 'B' AND ORGNR IN {sql_str}
"""
vof_for = pd.read_sql_query(sporring_for, conn)

fornummer = pd.Series(vof_for['FORETAKS_NR']).array
sql_str = lag_sql_str(fornummer)

sporring_bed = f"""
    SELECT FORETAKS_NR, ORGNR, NAVN, KARAKTERISTIKK, SN07_1, SB_TYPE
    FROM DSBBASE.SSB_BEDRIFT
    WHERE STATUSKODE = 'B' AND FORETAKS_NR IN {sql_str}
"""
vof_bdr = pd.read_sql_query(sporring_bed, conn)


/tmp/ipykernel_155/2550278207.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vof_for = pd.read_sql_query(sporring_for, conn)
/tmp/ipykernel_155/2550278207.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vof_bdr = pd.read_sql_query(sporring_bed, conn)


Henter organisasjons- og foretaksnummer fra Virksomhets- og foretaksregisteret (VoF) og samler disse i én tabell kalt ```vof```

In [35]:
vof_for = vof_for.rename(columns = {'NAVN':'NAVN_FORETAK'})
vof_for = vof_for.rename(columns = {'ORGNR':'ORGNR_FORETAK'})

vof_bdr = vof_bdr.rename(columns = {'ORGNR':'ORGNR_BEDRIFT'})
vof_bdr['KARAKTERISTIKK'] = vof_bdr['KARAKTERISTIKK'].fillna("")
vof_bdr['NAVN_BEDRIFT'] = vof_bdr['NAVN'] + " " + vof_bdr['KARAKTERISTIKK']

vof_bdr=vof_bdr.drop(columns=['NAVN', 'KARAKTERISTIKK'])

vof = pd.merge(vof_bdr,vof_for, how='left', on='FORETAKS_NR')
vof = vof.drop(columns=['FORETAKS_NR'])

rapporteringsenheter['ORGNR_FORETAK'] = rapporteringsenheter['ORGNR_FORETAK'].apply(str)
rapporteringsenheter_vof = pd.merge(vof,rapporteringsenheter, how='left', on='ORGNR_FORETAK')

In [36]:
rapporteringsenheter_vof.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1243 entries, 0 to 1242
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ORGNR_BEDRIFT  1243 non-null   object
 1   SN07_1         1243 non-null   object
 2   SB_TYPE        1243 non-null   object
 3   NAVN_BEDRIFT   1243 non-null   object
 4   ORGNR_FORETAK  1243 non-null   object
 5   NAVN_FORETAK   1243 non-null   object
 6   NAVN_KLASS     1243 non-null   object
 7   HELSEREGION    1243 non-null   object
 8   RHF            1206 non-null   object
dtypes: object(9)
memory usage: 97.1+ KB


## Dynarev

Henter ut SFU og tar senere vare på de som har har non-missing på ```SKJEMA_TYPE```.

In [37]:
sporring = f"""
    SELECT *
    FROM DSBBASE.DLR_ENHET_I_DELREG
    WHERE DELREG_NR IN ('24{siste_to_siffer_aargang}')
"""
SFU_data = pd.read_sql_query(sporring, conn)
print(f"Rader:    {SFU_data.shape[0]}\nKolonner: {SFU_data.shape[1]}")
SFU_data.info()


Rader:    1601
Kolonner: 127
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601 entries, 0 to 1600
Columns: 127 entries, DELREG_NR to STR_KODE
dtypes: datetime64[ns](4), float64(2), int64(1), object(120)
memory usage: 1.6+ MB


/tmp/ipykernel_155/2856274387.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SFU_data = pd.read_sql_query(sporring, conn)


In [38]:
# Enheter med verdi i 'KVITT_TYPE' filtreres ut (de er nedlagte enheter). Fjerner også enheter uten ORGNR
SFU_data = SFU_data[~SFU_data['KVITT_TYPE'].notnull()]
SFU_data = SFU_data[SFU_data['ORGNR'].notnull()]

In [39]:
def hent_data_delreg24x_og_19377x(x, skjema):
    sporring = f"""
        SELECT *
        FROM DYNAREV.VW_SKJEMA_DATA
        WHERE DELREG_NR IN ('24{x}', '19377{x}') AND SKJEMA IN ('{skjema}') AND AKTIV = '1'
    """
    sporring_df = pd.read_sql_query(sporring, conn)
    sporring_df = sporring_df[['SKJEMA','ENHETS_ID', 'FELT_ID', 'FELT_VERDI']]
    sporring_df = sporring_df.pivot(index=['ENHETS_ID','SKJEMA'], columns='FELT_ID', values='FELT_VERDI')
    return sporring_df

In [40]:
skjemadata38O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE38O')
skjemadata38P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE38P')
skjemadata39  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE39')
skjemadata40  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE40')
skjemadata41  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE41')
skjemadata44O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE44O')
skjemadata44P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE44P')
skjemadata45O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE45O')
skjemadata45P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE45P')
skjemadata46O = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, '46O')           # skj46 bytter navn for ulike aarganger (fiks med klass?)
skjemadata46P = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE46P')
skjemadata47  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE47')
skjemadata48  = hent_data_delreg24x_og_19377x(siste_to_siffer_aar_for, 'HELSE48')

/tmp/ipykernel_155/1139750569.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sporring_df = pd.read_sql_query(sporring, conn)
/tmp/ipykernel_155/1139750569.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sporring_df = pd.read_sql_query(sporring, conn)
/tmp/ipykernel_155/1139750569.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sporring_df = pd.read_sql_query(sporring, conn)
/tmp/ipykernel_155/1139750569.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or 

# Setter sammen master-SFU-fil med all nødvendig informasjon

SFU: orgnummer, orgnummer_foretak, foretaksnavn, helseregionnavn, helseregionsnummer
"alle virksomhetene til speshelse ligger i 2421"


In [41]:
SFUklass = pd.merge(SFU_data, rapporteringsenheter, how="left", on="ORGNR_FORETAK")

SFUklass['HELSEREGION'] = SFUklass['HELSEREGION'].fillna("06")

SFUklass['SKJEMA_TYPE'] = SFUklass['SKJEMA_TYPE'].apply(lambda x: str(x).split(" "))
SFUklass[['NAVN1', 'NAVN2', 'NAVN3', 'NAVN4', 'NAVN5']] = SFUklass[['NAVN1', 'NAVN2', 'NAVN3', 'NAVN4', 'NAVN5']].fillna("")

In [42]:
# Slår sammen alle navnekolonnene til en adskilt med ett mellomrom
SFUklass['NAVN'] = SFUklass['NAVN1'] + " " +\
                   SFUklass['NAVN2'] + " " +\
                   SFUklass['NAVN3'] + " " +\
                   SFUklass['NAVN4'] + " " +\
                   SFUklass['NAVN5']
SFUklass['NAVN'] = SFUklass['NAVN'].apply(lambda x: ' '.join(x.split()))

# Lager droplister
Sortert etter hva slags dropliste som skal lages.

In [43]:
# En liste over alle variabelnavn som peker til droplister
skjemaer_til_droplister = ["skj0X", "skj0Y", "skj38O", "skj38P", "skj39", "skj40", "skj41",
                           "skj44O", "skj44P", "skj45O", "skj45P", "skj46O",
                           "skj46P", "skj47", "skj48", "brukerliste_df"]

Funksjon som returnerer en tabell med alle rader som inneholder et gitt skjema:

In [44]:
# eks: 
# tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0X")
def tabell_som_inneholder_skjema(df, kol, skjemanavn):
    ant = df.shape[0]
    rader_med_skjemanavn = []
    for i in range(ant):
        if skjemanavn in df[kol].iloc[i]:
            rader_med_skjemanavn.append(i)
    return df.iloc[rader_med_skjemanavn]

## Skjemaer som hentes rett fra klass og SFU

### 0X (Resultatregnskap for RHF og HF)

In [45]:
# kolonner som skal være med i droplisten:
kolonner0X0Y404148 = ["USERID", "REGION_NR", "REGION_NAVN", "FORETAK_ORGNR", "FORETAK_NAVN"]

skj0X = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0X")
skj0X = skj0X.rename(columns = {'NAVN1':'FORETAK_NAVN',
                                'ORGNR':'USERID',
                                'HELSEREGION':'REGION_NR',
                                'RHF':'REGION_NAVN'})
skj0X['FORETAK_ORGNR'] = skj0X['USERID']

skj0X = skj0X[kolonner0X0Y404148].sort_values('REGION_NR')

### 0Y (Balanseregnskap for RHF og HF)

In [46]:
skj0Y = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0Y")
skj0Y = skj0Y.rename(columns = {'NAVN1':'FORETAK_NAVN', 
                                'ORGNR':'USERID', 
                                'HELSEREGION':'REGION_NR', 
                                'RHF':'REGION_NAVN'})
skj0Y['FORETAK_ORGNR'] = skj0Y['USERID']

skj0Y = skj0Y[kolonner0X0Y404148].sort_values('REGION_NR')

### skj40 (Kontantstrømoppstilling)

In [47]:
skj40 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "0Y")
skj40 = skj40.rename(columns = {'NAVN1':'FORETAK_NAVN',
                                'ORGNR':'USERID',
                                'HELSEREGION':'REGION_NR',
                                'RHF':'REGION_NAVN'})
skj40['FORETAK_ORGNR'] = skj40['USERID']

skj40 = skj40[kolonner0X0Y404148].sort_values('REGION_NR')

### skj41 (Private spesialister med driftsavtale)

In [48]:
skj41 = RHF_kode_klass.copy()
skj41 = skj41.rename(columns = {'HELSEREGION': 'REGION_NR',
                                'RHF': 'REGION_NAVN',
                                'ORGNR_FORETAK': 'FORETAK_ORGNR'})
skj41['USERID'] = skj41['FORETAK_ORGNR']
skj41['FORETAK_NAVN'] = skj41['REGION_NAVN']
skj41 = skj41[kolonner0X0Y404148]

### skj48 (Praksiskonsulentordningen)

In [49]:
skj48 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "48").copy()
skj48 = skj48[["NAVN_KLASS", "ORGNR_FORETAK", "HELSEREGION"]]
skj48 = pd.merge(skj48, RHF_kode_klass[['HELSEREGION', 'RHF']], how="left", on="HELSEREGION")
skj48 = pd.concat([skj48, HF])
skj48 = skj48.rename(columns={"NAVN_KLASS": "FORETAK_NAVN",
                              "ORGNR_FORETAK": "FORETAK_ORGNR",
                              "HELSEREGION": "REGION_NR",
                              "RHF": "REGION_NAVN"})
skj48['USERID'] = skj48['FORETAK_ORGNR']

skj48 = skj48[kolonner0X0Y404148] \
    .sort_values("REGION_NR") \
    .reset_index(drop=True)

## Skjemaer til offentlige foretak
Inneholder et visst antall kolonner med virksomheters navn og orgnr.

In [50]:
def lag_navn_orgnr_kolonnenavn(ant_kolonner):
    nvn, org = [], []
    for i in range(1,ant_kolonner+1):
        nvn.append(f'NAVN_VIRK{i}')
        org.append(f'ORGNR_VIRK{i}')
    return nvn, org

Denne funksjonen tar inn en dataframe med undervirksomheter som hører til et foretaksnummer og antall kolonner som skal være med i droplisten. Undervirksomhetene legges etterhverandre med riktig korrespondanse mellom navn- og orgnr-kolonner.

In [51]:
def lag_navn_orgnr_kolonner(fvdf, ant_kolonner, med_foretak = True):
    foretak_rader = fvdf.ORGNR_FORETAK.value_counts().index.to_numpy()              # tar ut unike foretaksnummer i en liste

    foretak_org_df = -1
    for nr in foretak_rader:                                                        # blar gjennom listen og lager kolonner til hvert foretak
        if med_foretak:
            orgnr_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}"')['ORGNR']
            navn_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}"')['NAVN']
        else:
            orgnr_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}" and ORGNR != "{nr}"')['ORGNR']
            navn_df = fvdf.query(f'ORGNR_FORETAK  == "{nr}" and ORGNR != "{nr}"')['NAVN']    

        ant_virksomheter = orgnr_df.shape[0]
        tom_df = pd.Series(np.nan, index=range(ant_kolonner-ant_virksomheter))
        orgnr_df = pd.concat([orgnr_df, tom_df], axis=0)
        orgnr_df.index = orgnr_virk

        ant_navn = navn_df.shape[0]
        tom_df = pd.Series(np.nan, index=range(ant_kolonner-ant_navn))
        navn_df = pd.concat([navn_df, tom_df], axis=0)
        navn_df.index = navn_virk

        foretak_df = pd.concat([orgnr_df, navn_df], axis=0)
        foretak_df['ORGNR_FORETAK'] = nr

        if type(foretak_org_df) == int:                                             # limer sammen hvert foretak med nye kolonner til en stor tabell
            foretak_org_df = foretak_df
        else:
            foretak_org_df = pd.concat([foretak_org_df, foretak_df], axis = 1)
    return foretak_org_df.transpose()

### skj38O (TSB for offentlige helseforetak)

Alle offentlige helseforetak foruten ```SUNNAAS```. Bruker ```HF```-tabellen laget i kapittel 2.1.1

In [52]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata38O[['FORETAKETS_ORGNR','DGN_DGN']].reset_index().copy()
                [['FORETAKETS_ORGNR','DGN_DGN']]
                .rename(columns={'DGN_DGN': 'D_PLAS_FJOR'}))

In [53]:
skj38O = HF.query('ORGNR_FORETAK != "883971752"').copy()                           # Tar ut SUNNAAS

In [54]:
# Lager kolonneoverskrifter i tråd med tidligere droplister:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(20)

onskede_kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
                    "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
                    "D_PLAS_FJOR"] + orgnr_virk + navn_virk

Henter data fra SFU med næringskode ("86.106") og statuskode ("B")
Næringskode i kolonne SN07_1

In [55]:
skj38O['tmp_bool'] = True

In [56]:
finne_virksomheter_df = pd.merge(SFUklass, skj38O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 == "86.106" and STATUS == "B"')
finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN']]

In [57]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df, 20)

In [58]:
skj38O = pd.merge(skj38O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

skj38O['USERID'] = skj38O['ORGNR_FORETAK']
skj38O = skj38O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

skj38O = pd.merge(skj38O, d_plass_fjor, how="left", on="FORETAKETS_ORGNR")

skj38O = skj38O[onskede_kolonner]

### skj44O (Psykisk helsevern for voksne PHFV)

OBS: Nye enheter 2022

Alle offentlige helseforetak foruten ```SUNNAAS```. Bruker ```HF```-tabellen laget i kapittel 2.1.1

In [59]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata44O[['FORETAKETS_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FORETAKETS_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [60]:
skj44O = HF.query('ORGNR_FORETAK != "883971752"').copy()                    # Tar ut SUNNAAS

In [61]:
# Lager kolonneoverskrifter i tråd med tidligere .csv-filer:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(20)

onskede_kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
                    "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
                    "D_PLAS_FJOR"] + orgnr_virk + navn_virk

Henter data fra SFU med næringskode ("86.104") og statuskode ("B"). Finner næringskoden i kolonne ```SN07_1```

In [62]:
skj44O['tmp_bool'] = True

In [63]:
finne_virksomheter_df = pd.merge(SFUklass, skj44O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 == "86.104" and STATUS == "B"')
finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN']]

In [64]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df, 20)

In [65]:
skj44O = pd.merge(skj44O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

skj44O['USERID'] = skj44O['ORGNR_FORETAK']
skj44O['USERID'] = skj44O['ORGNR_FORETAK']
skj44O = skj44O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

skj44O = pd.merge(skj44O, d_plass_fjor, how="left", on="FORETAKETS_ORGNR")

skj44O = skj44O[onskede_kolonner]

### skj45O (Psykisk helsevern for barn og unge (PHBU/BUP), offentlige helseforetak)

In [66]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata45O[['FORETAKETS_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FORETAKETS_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [67]:
skj45O = HF.query('ORGNR_FORETAK != "883971752"').copy()                    # Tar ut SUNNAAS

In [68]:
# Lager kolonneoverskrifter i tråd med tidligere .csv-filer:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(20)

onskede_kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
                    "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
                    "D_PLAS_FJOR"] + orgnr_virk + navn_virk

Henter data fra SFU med næringskode ("86.105") og statuskode ("B"). Finner næringskoden i kolonne ```SN07_1```

In [69]:
skj45O['tmp_bool'] = True

In [70]:
finne_virksomheter_df = pd.merge(SFUklass, skj45O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 == "86.105" and STATUS == "B"')
finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN']]

In [71]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df, 20)

In [72]:
skj45O = pd.merge(skj45O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

In [73]:
skj45O['USERID'] = skj45O['ORGNR_FORETAK']
skj45O['USERID'] = skj45O['ORGNR_FORETAK']
skj45O = skj45O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

In [74]:
skj45O = pd.merge(skj45O, d_plass_fjor, how="left", on="FORETAKETS_ORGNR")

In [75]:
skj45O = skj45O[onskede_kolonner]

### skj46O (Somatiske sykehus, offentlige helseforetak)
```SUNNAAS``` skal være med her

NB. Antall avsatte kolonner til undervirksomheter er 24.

In [76]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata46O[['ORGNR_VIRK1','SEN_HT', 'SDGN_HT']].reset_index().copy()
                [['ORGNR_VIRK1','SEN_HT', 'SDGN_HT']]
                .rename(columns={'SEN_HT': 'SEN_HT_FJOR',
                                 'SDGN_HT': 'SDGN_HT_FJOR',
                                 'ORGNR_VIRK1': 'FINST_ORGNR'}))

In [77]:
skj46O = HF.copy()

In [78]:
# Lager kolonneoverskrifter i tråd med tidligere .csv-filer:
navn_virk, orgnr_virk = lag_navn_orgnr_kolonnenavn(24)

kolonner = ["USERID", "HELSEREGION", "HELSEREGION_NAVN",
            "FORETAKETS_ORGNR", "FORETAKETS_NAVN",
            'SEN_HT_FJOR', 'SDGN_HT_FJOR'] + orgnr_virk + navn_virk

Henter data fra SFU med næringskoder ("86.101") ("86.102") ("86.103") ("86.107"). Finner næringskoden i kolonne ```SN07_1```

In [79]:
skj46O['tmp_bool'] = True

finne_virksomheter_df = pd.merge(SFUklass, skj46O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df = finne_virksomheter_df.query('tmp_bool == True and SN07_1 in ["86.101", "86.102", "86.103","86.107",]')

finne_virksomheter_df = finne_virksomheter_df[['ORGNR','ORGNR_FORETAK','NAVN', 'NAVN_KLASS']]

finne_virksomheter_df = pd.merge(finne_virksomheter_df, d_plass_fjor, how="left", left_on="ORGNR", right_on="FINST_ORGNR")

finne_virksomheter_df['SEN_HT_FJOR'] = pd.to_numeric(finne_virksomheter_df['SEN_HT_FJOR'])
finne_virksomheter_df['SDGN_HT_FJOR'] = pd.to_numeric(finne_virksomheter_df['SDGN_HT_FJOR'])

finne_virksomheter_df = finne_virksomheter_df.groupby(["ORGNR_FORETAK", "NAVN_KLASS"]).sum(numeric_only=True).reset_index()
finne_virksomheter_df = finne_virksomheter_df.rename(columns={"NAVN_KLASS": "FORETAKETS_NAVN"})

In [80]:
finne_virksomheter_df2 = pd.merge(SFUklass, skj46O, how="left", on=["ORGNR_FORETAK", "RHF", "NAVN_KLASS", "HELSEREGION"])
finne_virksomheter_df2 = finne_virksomheter_df2.query('tmp_bool == True and SN07_1 in ["86.101", "86.102", "86.103","86.107",]')

finne_virksomheter_df2 = finne_virksomheter_df2[['ORGNR','ORGNR_FORETAK','NAVN', 'NAVN_KLASS']]

In [81]:
undervirksomheter_navn_og_kolonner = lag_navn_orgnr_kolonner(finne_virksomheter_df2, 24, False)

skj46O = pd.merge(skj46O, undervirksomheter_navn_og_kolonner, how="left", on="ORGNR_FORETAK")

skj46O = pd.merge(skj46O, finne_virksomheter_df, how="left", on="ORGNR_FORETAK")

skj46O['USERID'] = skj46O['ORGNR_FORETAK']
skj46O = skj46O.rename(columns={"ORGNR_FORETAK": "FORETAKETS_ORGNR",
                                "NAVN_KLASS": "FORETAKETS_NAVN",
                                "RHF": "HELSEREGION_NAVN"})

In [82]:
skj46O = skj46O[kolonner]

## Skjemaer til private foretak og deres underinstitusjoner
Droplisten inneholder kolonne med rapporteringspliktige underinstitusjoner i en kolonne atskilt med \n

#### Nyttige funksjoner som skal gjøre koden mer lesbar

In [83]:
def hent_foretaksnavn_til_virksomhetene_fra_SFU(unike_foretak):
    unike_foretak = pd.DataFrame(unike_foretak)
    unike_foretak.columns = ["ORGNR_FORETAK"]

    finn_foretaksnavn = pd.merge(unike_foretak, SFUklass[["ORGNR", "NAVN"]],
                                 how="left",
                                 left_on="ORGNR_FORETAK",
                                 right_on="ORGNR")
    finn_foretaksnavn = finn_foretaksnavn[["ORGNR_FORETAK", "NAVN"]]\
                                        .rename(columns={"NAVN": "FORETAK_NAVN",
                                        "ORGNR_FORETAK": "FORETAK_ORGNR"})
    return finn_foretaksnavn

Lager kolonnen INSTLIST med riktig antall \n for at det skal passe med skjema. Passer på at rapporteringsenheten kommer først i listen. (Enhet som har H_VAR1_A lik orgnummer)

In [84]:
def instlist_med_riktig_antall_n(finst_orgnr_df):
    df_til_instlist = pd.merge(SFUklass, finst_orgnr_df, left_on="H_VAR1_A", right_on="FINST_ORGNR")
    df_til_instlist['rapporterer_til_annen_virksomhet'] = (df_til_instlist.ORGNR != df_til_instlist.H_VAR1_A)
    df_til_instlist = df_til_instlist[['NAVN', 'ORGNR', "FINST_ORGNR", 'rapporterer_til_annen_virksomhet']]
    df_til_instlist['orgnr_navn'] = df_til_instlist['ORGNR'] + " - " + df_til_instlist['NAVN']

    rapporterer_ikke_til_annen_v = df_til_instlist.query('~rapporterer_til_annen_virksomhet')[['ORGNR', 'orgnr_navn']]
    rapporterer_til_annen_v = df_til_instlist.query('rapporterer_til_annen_virksomhet')[["FINST_ORGNR", 'orgnr_navn']]
    
    # mellomsteg for å legge institusjonene horisontalt ved siden av rapporteringsnummeret
    # itererer over alle unike rapporteringsnummer
    tmpdf = -1
    for finst_orgnr in rapporterer_ikke_til_annen_v.ORGNR.unique():
        instliste = "\\n".join(rapporterer_til_annen_v.query(f'FINST_ORGNR=="{finst_orgnr}"').orgnr_navn)
        if len(instliste) > 0:
            if type(tmpdf) == int:
                tmpdf = pd.DataFrame([finst_orgnr, instliste])
            else:
                nytt_tillegg = pd.DataFrame([finst_orgnr, instliste])
                tmpdf = pd.concat([tmpdf, nytt_tillegg], axis=1)
    
    if type(tmpdf) != int:
        tmpdf = tmpdf.transpose()
        tmpdf.columns = ['FINST_ORGNR', 'INSTLISTE_HALE']   
    
    rapporterer_ikke_til_annen_v = rapporterer_ikke_til_annen_v.rename(columns={'ORGNR': 'FINST_ORGNR'})
    return rapporterer_ikke_til_annen_v, tmpdf

In [85]:
def legg_paa_hale_med_n(df):
    df['INSTLISTE_HALE'] = df['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
    df['antall_n_mangler'] = df['INSTLISTE_HALE'].apply(lambda x: 11-x.count("\\n"))
    df['INSTLIST'] = df.apply(lambda x: x.orgnr_navn + \
                                                "\\n" + \
                                                x.INSTLISTE_HALE + \
                                                x.antall_n_mangler * ("\\n"), axis=1)
    return df

In [86]:
kolonner_i_alle_private = ['USERID',
                            'REGION_NR',
                            'REGION_NAVN',
                            'FORETAK_ORGNR',
                            'FORETAK_NAVN',
                            'FINST_ORGNR',
                            'FINST_NAVN']

### skj38P (TSB for private helseforetak)

Henter inn alle rader i SFU som har skjematype 381. Variabelen ```H_VAR1_A```  sier hvilket orgnummer foretaket rapporterer til. Koden under bruker denne til å variabelen til å plassere rapporteringsenhetene først, og alle deres underinstitusjoner i variabelen ```INSTLIST```. 

For at filen skal ha riktig format, skal det tilsammen være 13 institusjoner per foretak i ```INSTLIST```. Fyller automatisk ut linjeskift ```\n``` slik at antallet blir 13.

In [87]:
# kolonner som skal være med i droplisten:
kolonner = kolonner_i_alle_private +\
            ['INSTLIST']

skj38P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "381").copy()
skj38P = skj38P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj38P['USERID'] = skj38P['FORETAK_ORGNR']


In [88]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj38P = pd.merge(skj38P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj38P['REGION_NAVN'] = skj38P['RHF']
skj38P.REGION_NAVN = skj38P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [89]:
foretaksnavn = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj38P.FORETAK_ORGNR.unique())
skj38P = pd.merge(skj38P, foretaksnavn, how="left", on="FORETAK_ORGNR")

In [90]:
rapporteringsenhet, undervirksomheter = instlist_med_riktig_antall_n(pd.DataFrame(skj38P['FINST_ORGNR']))

In [91]:
skj38P = pd.merge(skj38P, rapporteringsenhet, how="left", on="FINST_ORGNR")
skj38P = pd.merge(skj38P, undervirksomheter, how="left", on="FINST_ORGNR")

skj38P = legg_paa_hale_med_n(skj38P)

In [92]:
skj38P = skj38P[kolonner]

### skj39 (Resultatregnskap for private helseforetak)

Lages på nesten samme måte som skjema 38P, med unntak av at man henter ut SKJEMA_TYPE 39 fra SFU og legger til kolonnen ```INSTTYPE``` med institusjonstype hentet fra hjelpevariabelen ```SN07_1```.

In [93]:
# kolonner som skal være med i droplisten:
kolonner = kolonner_i_alle_private +\
           ['INSTTYPE', 'INSTLIST']

skj39 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "39").copy()
skj39 = skj39.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj39['USERID'] = skj39['FORETAK_ORGNR']


In [94]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj39 = pd.merge(skj39, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj39['REGION_NAVN'] = skj39['RHF']
skj39.REGION_NAVN = skj39.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [95]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj39.FORETAK_ORGNR.unique())

skj39 = pd.merge(skj39, foretak, how="left", on="FORETAK_ORGNR")

In [96]:
rapporteringsenhet, undervirksomheter = instlist_med_riktig_antall_n(pd.DataFrame(skj39['FINST_ORGNR']))

In [97]:
skj39 = pd.merge(skj39, rapporteringsenhet, how="left", on="FINST_ORGNR")
skj39 = pd.merge(skj39, undervirksomheter, how="left", on="FINST_ORGNR")
skj39 = legg_paa_hale_med_n(skj39)

In [98]:
institusjonstype = {
    451: 'Psykisk helsevern for barn og unge',
    461: 'Somatiske sykehus',
    381: 'Rusmiddelinstitusjoner',
    441: 'Psykisk helsevern for voksne',
    47:  'Somatiske rehab.-og opptr.inst.'
}

skj39 = skj39.rename(columns={'H_VAR2_N': 'INSTTYPE'})

In [99]:
skj39['INSTTYPE'] = skj39['INSTTYPE'].map(institusjonstype)

In [100]:
# Hvis FORETAK_NAVN er tom, bruk NAVN1 fra SFU:
skj39.loc[skj39['FORETAK_NAVN'].isnull(),'FORETAK_NAVN'] = skj39['NAVN1']

In [101]:
# Tar kun vare på de kolonnene jeg spesifiserer i begynnelsen
skj39 = skj39[kolonner]

### skj44P (Psykisk helsevern for voksne (PHFV), private helseforetak)

In [102]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata44P[['FINST_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FINST_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [103]:
# kolonner som skal være med i droplisten:
kolonner = kolonner_i_alle_private +\
           ['INSTLIST', 'D_PLAS_FJOR']

skj44P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "441").copy()
skj44P = skj44P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj44P['USERID'] = skj44P['FORETAK_ORGNR']


In [104]:
# Importerer riktig regionnummer fra KLASS og gir foretak 
# som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj44P = pd.merge(skj44P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj44P['REGION_NAVN'] = skj44P['RHF']
skj44P.REGION_NAVN = skj44P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [105]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj44P.FORETAK_ORGNR.unique())

skj44P = pd.merge(skj44P, foretak, how="left", on="FORETAK_ORGNR")

In [106]:
rapporteringsenhet, undervirksomheter = instlist_med_riktig_antall_n(pd.DataFrame(skj44P['FINST_ORGNR']))

In [107]:
skj44P = pd.merge(skj44P, rapporteringsenhet, how="left", on="FINST_ORGNR")
skj44P = pd.merge(skj44P, undervirksomheter, how="left", on="FINST_ORGNR")
skj44P = legg_paa_hale_med_n(skj44P)

In [108]:
skj44P = pd.merge(skj44P, d_plass_fjor, how="left", on="FINST_ORGNR")

In [109]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen 
skj44P = skj44P[kolonner]

### skj45P (Psykisk helsevern for barn og unge (PHBU/BUP), private helseforetak)

In [110]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata45P[['FINST_ORGNR','D_PLAS_T']].reset_index().copy()
                [['FINST_ORGNR','D_PLAS_T']]
                .rename(columns={'D_PLAS_T': 'D_PLAS_FJOR'}))

In [111]:
# kolonner som skal være med i droplisten:
kolonner = kolonner_i_alle_private +\
           ['INSTLIST', 'D_PLAS_FJOR']

skj45P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "451").copy()
skj45P = skj45P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj45P['USERID'] = skj45P['FORETAK_ORGNR']

In [112]:
# Importerer riktig regionnummer fra KLASS og gir foretak 
# som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj45P = pd.merge(skj45P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj45P['REGION_NAVN'] = skj45P['RHF']
skj45P.REGION_NAVN = skj45P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [113]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj45P.FORETAK_ORGNR.unique())

skj45P = pd.merge(skj45P, foretak, how="left", on="FORETAK_ORGNR")

In [114]:
rapporteringsenhet, undervirksomheter = instlist_med_riktig_antall_n(pd.DataFrame(skj45P['FINST_ORGNR']))

In [115]:
skj45P = pd.merge(skj45P, rapporteringsenhet, how="left", on="FINST_ORGNR")
if type(undervirksomheter) != int:
    skj45P = pd.merge(skj45P, undervirksomheter, how="left", on="FINST_ORGNR")
    skj45P['INSTLISTE_HALE'] = skj45P['INSTLISTE_HALE'].fillna("\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n")
else:
    skj45P['INSTLISTE_HALE'] = "\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n"


skj45P = legg_paa_hale_med_n(skj45P)

In [116]:
skj45P = pd.merge(skj45P, d_plass_fjor, how="left", on="FINST_ORGNR")

In [117]:
# Tar kun vare på de kolonnene spesifisert i begynnelsen
skj45P = skj45P[kolonner]

### skj46P (Somatiske sykehus, private helseforetak)

In [118]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata46P[['FINST_ORGNR','SEN_HT', 'SDGN_HT']].reset_index().copy()
                [['FINST_ORGNR','SEN_HT', 'SDGN_HT']]
                .rename(columns={'SEN_HT': 'SEN_HT_FJOR',
                                 'SDGN_HT': 'SDGN_HT_FJOR'}))

In [119]:
# kolonner som skal være med i droplisten:
kolonner = kolonner_i_alle_private +\
            ['SEN_HT_FJOR', 'SDGN_HT_FJOR']

skj46P = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "461").copy()
skj46P = skj46P.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj46P['USERID'] = skj46P['FORETAK_ORGNR']


In [120]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj46P = pd.merge(skj46P, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj46P['REGION_NAVN'] = skj46P['RHF']
skj46P.REGION_NAVN = skj46P.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [121]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj46P.FORETAK_ORGNR.unique())

skj46P = pd.merge(skj46P, foretak, how="left", on="FORETAK_ORGNR")

In [122]:
rapporteringsenhet, undervirksomheter = instlist_med_riktig_antall_n(pd.DataFrame(skj46P['FINST_ORGNR']))

In [123]:
skj46P = pd.merge(skj46P, rapporteringsenhet, how="left", on="FINST_ORGNR")

In [124]:
skj46P = pd.merge(skj46P, undervirksomheter, how="left", on="FINST_ORGNR")
skj46P.orgnr_navn = skj46P.orgnr_navn.fillna("")

skj46P = legg_paa_hale_med_n(skj46P)

In [125]:
skj46P = pd.merge(skj46P, d_plass_fjor, how="left", on="FINST_ORGNR")

In [126]:
skj46P.loc[skj46P.FORETAK_NAVN.isnull(), 'FORETAK_NAVN'] = skj46P.NAVN1

In [127]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen 
skj46P = skj46P[kolonner]

### skj47 (Somatiske institusjoner)

In [128]:
# Henter døgnplasser fra foregående år
d_plass_fjor = (skjemadata47[['FINST_ORGNR','D_HELD', 'SDGN_SUM']].reset_index().copy()
                [['FINST_ORGNR','D_HELD', 'SDGN_SUM']]
                .rename(columns={'D_HELD': 'D_HELD_IFJOR',
                                 'SDGN_SUM': 'SDGN_SUM_IFJOR'}))

In [129]:
# kolonner som skal være med i droplisten:
kolonner = kolonner_i_alle_private +\
           ['D_HELD_IFJOR','SDGN_SUM_IFJOR']

skj47 = tabell_som_inneholder_skjema(SFUklass, 'SKJEMA_TYPE', "47").copy()
skj47 = skj47.rename(columns = {"ORGNR_FORETAK": "FORETAK_ORGNR",
                                   "NAVN": "FINST_NAVN",
                                   'ORGNR':'FINST_ORGNR',
                                   'HELSEREGION':'REGION_NR',
                                   'RHF':'REGION_NAVN'})

# USERID er alltid foretaksnummer
skj47['USERID'] = skj47['FORETAK_ORGNR']

In [130]:
# Importerer riktig regionnummer fra KLASS og gir foretak som ikke er offentlige betegnelsen "PRIVATE INSTITUSJONER"
skj47 = pd.merge(skj47, RHF_kode_klass[["HELSEREGION", "RHF"]], how="left" , left_on="REGION_NR", right_on="HELSEREGION")

skj47['REGION_NAVN'] = skj47['RHF']
skj47.REGION_NAVN = skj47.REGION_NAVN.fillna("PRIVATE INSTITUSJONER")

In [131]:
# Henter foretaksnavn til virksomhetene fra SFU
foretak = hent_foretaksnavn_til_virksomhetene_fra_SFU(skj47.FORETAK_ORGNR.unique())
skj47 = pd.merge(skj47, foretak, how="left", on="FORETAK_ORGNR")

In [132]:
rapporteringsenhet, undervirksomheter = instlist_med_riktig_antall_n(pd.DataFrame(skj47['FINST_ORGNR']))

In [133]:
skj47 = pd.merge(skj47, rapporteringsenhet, how="left", on="FINST_ORGNR")
skj47 = pd.merge(skj47, undervirksomheter, how="left", on="FINST_ORGNR")
skj47 = legg_paa_hale_med_n(skj47)

In [134]:
skj47 = pd.merge(skj47, d_plass_fjor, how="left", on="FINST_ORGNR")

In [135]:
# Hvis FORETAK_NAVN er tom, bruk NAVN1 fra SFU:
skj47.loc[skj47['FORETAK_NAVN'].isnull(),'FORETAK_NAVN'] = skj47['NAVN1']

In [136]:
# Tar kun vare på de kolonnene jeg spesifiserte i begynnelsen
skj47 = skj47[kolonner]

# Brukerlister
Dette er en oversikt over alle foretak som skal ha skjemaer og hvilke skjemaer de skal få. Også kalt _populasjonsliste_ av IT-KOSTRA.

- Alle hjelpeforetak som skal rapportere skal ha skjema: `"0X 0Y 40"`
    - Dette ser ut til å være alle som ikke er apotek. Foretaksnummerne til disse er listet opp under.
- Alle RHF skal ha skjema: `"0X 0Y 40 41"`
- Alle HF skal ha skjema: `"0X 0Y 40 380 440 450 460 48"`
- Alle foretak med oppdrag og bestillerdokument skal ha skjema: `"39 381 441 451 461 47 48"`
- Alle private foretak skal ha skjema: `"39 381 441 451 461 47"`

## Hjelpeforetak

In [137]:
hjelpeforetak_som_ikke_er_rapporteringsenhet = ['918098275', '983974716', '983974805', '983974937', '992281618']

In [138]:
# tar ut RHFene fra listen
df_hj = regfel_df.query("level != '1' and parentCode not in ['03','04','05','12']") 

# tar bort unntakene
df_hj = df_hj[~df_hj['code'].isin(hjelpeforetak_som_ikke_er_rapporteringsenhet)]

In [139]:
df_hj['s'] = "0X 0Y 40"
df_hj = df_hj[['name', 'code', 's']]
df_hj.columns = ['FORETAK_NAVN', 'ORGNR_FORETAK', 'SKJEMA_TYPE']

## RHF

In [140]:
df_rhf = regfel_df[regfel_df['parentCode'].isin(['03','04','05','12'])].copy()

df_rhf['s'] = "0X 0Y 40 41"
df_rhf = df_rhf[['name', 'code', 's']]
df_rhf.columns = ['FORETAK_NAVN', 'ORGNR_FORETAK', 'SKJEMA_TYPE']

## HF

In [141]:
df_hf = HF.copy()

In [142]:
df_hf['s'] = "0X 0Y 40 380 440 450 460 48"
df_hf = df_hf[['NAVN_KLASS', 'ORGNR_FORETAK', 's']]
df_hf.columns = ['FORETAK_NAVN', 'ORGNR_FORETAK', 'SKJEMA_TYPE']

## Private

In [143]:
df_p = SFUklass[SFUklass['H_VAR2_A'] == 'PRIVAT'].copy()

df_p['s'] = "39 381 441 451 461 47"
df_p['NYTT_NAVN'] = df_p['NAVN1'] + " " + df_p['NAVN2']
df_p = df_p[['NYTT_NAVN', 'ORGNR_FORETAK', 's']]
df_p.columns = ['FORETAK_NAVN', 'ORGNR_FORETAK', 'SKJEMA_TYPE']
print("Med duplikater: ", df_p.shape)

df_p = df_p.drop_duplicates(subset=['ORGNR_FORETAK'], keep='first')
print("Uten duplikater: ", df_p.shape)


(166, 3)
(96, 3)


## Oppdrags- og bestillerdokument

In [144]:
df_op = SFUklass[SFUklass['H_VAR2_A'] == 'OPPDRAG'].copy()

df_op['s'] = "39 381 441 451 461 47 48"
df_op['NYTT_NAVN'] = df_op['NAVN1'] + " " + df_op['NAVN2']
df_op = df_op[['NYTT_NAVN', 'ORGNR_FORETAK', 's']]
df_op.columns = ['FORETAK_NAVN', 'ORGNR_FORETAK', 'SKJEMA_TYPE']
print("Med duplikater: ", df_op.shape)
df_op = df_op.drop_duplicates(subset=['ORGNR_FORETAK'], keep='first')
print("Uten duplikater: ", df_op.shape)

(42, 3)
(12, 3)


## Slå sammen til en dataframe som eksporteres til `.csv`

In [154]:
brukerliste_dfs = [df_hj, df_rhf, df_hf, df_p, df_op]
brukerliste_df = pd.concat(brukerliste_dfs)


(142, 3)


,FORETAK_NAVN,ORGNR_FORETAK,SKJEMA_TYPE
3,AKERSHUS UNIVERSITETSSYKEHUS HF,983971636,0X 0Y 40 380 440 450 460 48
56,ALERIS FROGNER,943545634,39 381 441 451 461 47
338,AVONOVA HELSE AS,884039312,39 381 441 451 461 47
36,BETANIEN SPESIALISTPOLIKLINIKK,922716552,39 381 441 451 461 47 48
64,BLÅ KORS BORGESTADKLINIKKEN SA,971450231,39 381 441 451 461 47
195,BLÅ KORS HAUGALAND A-SENTER,991314113,39 381 441 451 461 47
273,BLÅ KORS KLINIKK RUS OG AVHENGIGHET TJELDSUND AS,976692888,39 381 441 451 461 47
503,BLÅ KORS LADE BEHANDLINGSSENTER SA,892237832,39 381 441 451 461 47
132,BLÅ KORS ØST AS,977203287,39 381 441 451 461 47
189,CAPIO ANOREKSI SENTER AS,980524493,39 381 441 451 461 47


# Eksportering til .csv-filer og rapport

In [146]:
def rapport(skj):
    rader = eval(skj).shape[0]
    kolonner = eval(skj).shape[1]
    rader_med_missing = rader - eval(skj).dropna().shape[0]
    skl = 100 * "-"
    s = f"{skl}\n{skj}\t Rader: {rader} \t Kolonner: {kolonner} \t Rader som inneholder minst en missing value: {rader_med_missing} "
    return s

def lagre_dropliste_csv(skj):
    filnavn = "Dropliste_" + skj + "_" + str(aargang) + "_" + dato_idag + ".csv"
    if skj == "brukerliste_df":
        filnavn = "Brukerliste" + "_" + str(aargang) + "_" + dato_idag + ".csv"
    eval(skj).to_csv(sti_til_lagring + filnavn, sep=";", encoding='latin1', index=False)
    print(filnavn, " lagret")

In [147]:
for x in skjemaer_til_droplister:
    print(rapport(x))

----------------------------------------------------------------------------------------------------
skj0X	 Rader: 34 	 Kolonner: 5 	 Rader som inneholder minst en missing value: 0 
----------------------------------------------------------------------------------------------------
skj0Y	 Rader: 34 	 Kolonner: 5 	 Rader som inneholder minst en missing value: 0 
----------------------------------------------------------------------------------------------------
skj38O	 Rader: 19 	 Kolonner: 46 	 Rader som inneholder minst en missing value: 19 
----------------------------------------------------------------------------------------------------
skj38P	 Rader: 35 	 Kolonner: 8 	 Rader som inneholder minst en missing value: 1 
----------------------------------------------------------------------------------------------------
skj39	 Rader: 144 	 Kolonner: 9 	 Rader som inneholder minst en missing value: 0 
-------------------------------------------------------------------------------------

In [148]:
if lagre_filer:
    for x in skjemaer_til_droplister:
        lagre_dropliste_csv(x)